<a href="https://colab.research.google.com/github/MoniaK007/Redi/blob/main/perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Perceptron

Simple replica of a perceptron
(https://de.wikipedia.org/wiki/K%C3%BCnstliches_Neuron)

with transfer- and output-funktion

In [ ]:
from typing import List
import random
import math

class Perceptron:
    def __init__(self, input_count: int) -> None:
        self.weights = [random.uniform(-1, 1) for _ in range(input_count)]
        self.activation = self.relu 
    
    # sum all inputs and apply bias
    def transfer(self, x: List[float]) -> float:
        return sum(x[i] * self.weights[i] for i in range(len(x)))
    
    # apply activation function
    def output(self, net_output: float) -> float:
        return self.activation(net_output)

    # bias + weights
    def setWeights(self, new_weights):
        self.weights = new_weights

    def getWeights(self):
        return self.weights

    def __str__(self) -> str:
        return "weights: " + str(self.weights)
        
    #    
    # The following are activation functions:
    #
    def hard_limit(self, in_value: float) -> float:
        return 1.0 if in_value >= 0 else 0.0

    def piecewise_linear(self,in_value: float) -> float:
        if in_value >= 0.5:
            return 1
        elif in_value <= -0.5:
            return 0
        else:
            return in_value + 0.5

    def linear(self, in_value: float) -> float:
        return in_value

    def relu(self, in_value: float) -> float:
        return max(0, float(in_value))

    def sigmoid(self, in_value: float) -> float:
        return 1 / (1 + math.exp(-1 * in_value * 10))

    

        

## logic gate implementation using a perceptron

Some logic gates with preset weights

In [ ]:
import numpy as np

# NOT - GATE
negation = Perceptron(1)
negation.setWeights([1, -1])

print("NOT(0) = {}".format(negation.output(negation.transfer([1, 0]))))
print("NOT(1) = {}".format(negation.output(negation.transfer([1, 1]))))

In [ ]:
# AND - GATE
conjunction = Perceptron(2)
conjunction.setWeights([-1.5, 1.0, 1.0])

print("AND(0, 0) = {}".format(conjunction.output(conjunction.transfer([1, 0, 0]))))
print("AND(0, 1) = {}".format(conjunction.output(conjunction.transfer([1, 0, 1]))))
print("AND(1, 0) = {}".format(conjunction.output(conjunction.transfer([1, 1, 0]))))
print("AND(1, 1) = {}".format(conjunction.output(conjunction.transfer([1, 1, 1]))))

In [ ]:
import matplotlib.pyplot as plt
a = np.zeros((10,10), np.float32)
for x in range(0,10):
  for y in range(0,10):
    a[x][y] = conjunction.output(conjunction.transfer([x/10, y/10]))

plt.imshow(a, cmap='hot', interpolation='nearest')
plt.show()



In [ ]:
# OR - GATE

disjunction = Perceptron(2)
disjunction.setWeights([-0.5, 1.0, 1.0])

print("OR(0, 0) = {}".format(disjunction.output(disjunction.transfer([1, 0, 0]))))
print("OR(0, 1) = {}".format(disjunction.output(disjunction.transfer([1, 0, 1]))))
print("OR(1, 0) = {}".format(disjunction.output(disjunction.transfer([1, 1, 0]))))
print("OR(1, 1) = {}".format(disjunction.output(disjunction.transfer([1, 1, 1]))))



In [ ]:
import matplotlib.pyplot as plt
a = np.zeros((10,10), np.float32)
for x in range(0,10):
  for y in range(0,10):
    a[x][y] = disjunction.output(disjunction.transfer([x/10, y/10]))

plt.imshow(a, cmap='hot', interpolation='nearest')
plt.show()

## Training the perceptron weights

Now we let the Perceptron find the weights itself by using a learning algorithm.

In [ ]:
import numpy as np

class PerceptronTrainer:

    class GateInputOutput:
        def __init__(self, inputs, expectedOutputs):
            self.inputs = inputs
            self.expectedOutputs = expectedOutputs

    # hard coded truth tables for some logic gates:
    oneInput = np.array([
        [1, 0], 
        [1, 1]])
    NOT_GATE = GateInputOutput(oneInput, np.array([1, 0]))

    twoInputs = np.array([
        [1, 0, 0], 
        [1, 0, 1], 
        [1, 1, 0], 
        [1, 1, 1]])
    AND_GATE = GateInputOutput(twoInputs, np.array([0, 0, 0, 1]))
    NAND_GATE = GateInputOutput(twoInputs, np.array([1, 1, 1, 0]))
    OR_GATE = GateInputOutput(twoInputs, np.array([0, 1, 1, 1]))
    XOR_GATE = GateInputOutput(twoInputs, np.array([0, 1, 1, 0]))

    # #### TWIDDLE AREA: ¯\_(ツ)_/¯ ###############################################
    selectedInputOutput = AND_GATE
    ITERATIONS = 30
    LEARNING_RATE = 0.0137
    ACTIVATION = Perceptron.relu
    # #### :TWIDDLE AREA END ;) ###################################################

    inputs = selectedInputOutput.inputs
    expectedOutputs = selectedInputOutput.expectedOutputs

    def __init__(self):
        pass

    def train(self, logicGate, input, expectedOutputs, learningRate):
        weights = logicGate.getWeights()
        newWeights = np.zeros(input.shape[1])
        for idxN in range(input.shape[1]):
            newWeights[idxN] = weights[idxN] + self.sumOfErrors(logicGate, input, expectedOutputs, learningRate, idxN)
        #print("newWeights: ", newWeights)
        logicGate.setWeights(newWeights)

    def sumOfErrors(self, logicGate, input, expectedOutputs, learningRate, idxN):
        sumOfErrors = 0
        for row in range(input.shape[0]):
            outputs = logicGate.output(logicGate.transfer(input[row]))
            #print("outputs", outputs)
            sumOfErrors += learningRate * (expectedOutputs[row] - outputs) * input[row][idxN]
        return sumOfErrors

    def calcOutput(self, logicGate, inputs, expectedOutput):
        output = np.zeros(inputs.shape[0])
        errSum = 0
        for r in range(inputs.shape[0]):
            input = inputs[r]
            output[r] = logicGate.output(logicGate.transfer(input))
            errSum += abs(expectedOutput[r] - output[r])
        print("avg. error:", (errSum/ len(inputs)));
        return output


In [ ]:
trainer = PerceptronTrainer()
logicGate = Perceptron(len(trainer.inputs[0]))
logicGate.activation = trainer.ACTIVATION;

print("initial Perceptron configuration: " + str(logicGate))
for i in range(0,trainer.ITERATIONS):
    trainer.train(logicGate, trainer.inputs, trainer.expectedOutputs, trainer.LEARNING_RATE)

print("Perceptron configuration: ", logicGate)
trainer.calcOutput(logicGate, trainer.inputs, trainer.expectedOutputs)

In [ ]:
#@title
import matplotlib.pyplot as plt
a = np.zeros((11,11), np.float32)
for x in range(0,11):
  for y in range(0,11):
    a[x][y] = logicGate.output(logicGate.transfer([x/10, y/10]))

plt.imshow(a, cmap='hot', interpolation='nearest')
plt.show()